# Imports

In [1]:
!pip install pandas
import numpy as np
import pandas as pd
from numpy import genfromtxt
import matplotlib.pyplot as plt
import ast
import json

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import gc 
gc.collect()

4

In [4]:
import statistics as stat

# Chargement des données

Ici pour l'instant on teste sur une version raccourcie du jeu de données sinon c'est trop long. 

In [3]:
# Opening JSON file small version
#with open('train_porto_tokenized_small_1st_partV2.json', 'r') as openfile:
# Opening JSON file full version
with open('train_porto_tokenized.json', 'r') as openfile:

 
    # Reading from json file
    json_loaded = json.load(openfile)
 
# print(json_loaded)
print(type(json_loaded))

<class 'dict'>


On met en dataframe les données chargées.

In [5]:
data_loaded = pd.DataFrame(data=json_loaded)

On vérifie la tête de nos données et leur type :

In [6]:
data_loaded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1710670 entries, 0 to 1710669
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   TRIP_ID       int64  
 1   CALL_TYPE     object 
 2   ORIGIN_CALL   float64
 3   ORIGIN_STAND  float64
 4   TAXI_ID       int64  
 5   TIMESTAMP     int64  
 6   DAY_TYPE      object 
 7   MISSING_DATA  bool   
 8   POLYLINE      object 
 9   Tokenization  object 
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 132.1+ MB


In [7]:
#type(data_loaded['POLYLINE']), data_loaded['POLYLINE']

(pandas.core.series.Series,
 0          [[-8.618643, 41.141412], [-8.618499, 41.141376...
 1          [[-8.639847, 41.159826], [-8.640351, 41.159871...
 2          [[-8.612964, 41.140359], [-8.613378, 41.14035]...
 3          [[-8.574678, 41.151951], [-8.574705, 41.151942...
 4          [[-8.645994, 41.18049], [-8.645949, 41.180517]...
                                  ...                        
 1710665    [[-8.612469, 41.14602], [-8.612487, 41.145993]...
 1710666    [[-8.610138, 41.140845], [-8.610174, 41.140935...
 1710667                                                   []
 1710668    [[-8.630712, 41.154885], [-8.63073, 41.154813]...
 1710669    [[-8.615538, 41.140629], [-8.615421, 41.140746...
 Name: POLYLINE, Length: 1710670, dtype: object)

In [8]:
#type(data_loaded['POLYLINE'][0]), data_loaded['POLYLINE'][0] 

(list,
 [[-8.618643, 41.141412],
  [-8.618499, 41.141376],
  [-8.620326, 41.14251],
  [-8.622153, 41.143815],
  [-8.623953, 41.144373],
  [-8.62668, 41.144778],
  [-8.627373, 41.144697],
  [-8.630226, 41.14521],
  [-8.632746, 41.14692],
  [-8.631738, 41.148225],
  [-8.629938, 41.150385],
  [-8.62911, 41.151213],
  [-8.629128, 41.15124],
  [-8.628786, 41.152203],
  [-8.628687, 41.152374],
  [-8.628759, 41.152518],
  [-8.630838, 41.15268],
  [-8.632323, 41.153022],
  [-8.631144, 41.154489],
  [-8.630829, 41.154507],
  [-8.630829, 41.154516],
  [-8.630829, 41.154498],
  [-8.630838, 41.154489]])

In [9]:
#type(data_loaded['POLYLINE'][0][0] ), data_loaded['POLYLINE'][0][0]  

(list, [-8.618643, 41.141412])

In [10]:
#type(data_loaded['POLYLINE'][0][0][0] ), data_loaded['POLYLINE'][0][0][0]  

(float, -8.618643)

In [11]:
#type(data_loaded['Tokenization'][0][0] ), data_loaded['Tokenization'][0][0] 

(str, '8a7b63adb347fff')

In [12]:
data_loaded.head()

TRIP_ID CALL_TYPE  ORIGIN_CALL  ORIGIN_STAND   TAXI_ID  \
0  1372636858620000589         C          NaN           NaN  20000589   
1  1372637303620000596         B          NaN           7.0  20000596   
2  1372636951620000320         C          NaN           NaN  20000320   
3  1372636854620000520         C          NaN           NaN  20000520   
4  1372637091620000337         C          NaN           NaN  20000337   

    TIMESTAMP DAY_TYPE  MISSING_DATA  \
0  1372636858        A         False   
1  1372637303        A         False   
2  1372636951        A         False   
3  1372636854        A         False   
4  1372637091        A         False   

                                            POLYLINE  \
0  [[-8.618643, 41.141412], [-8.618499, 41.141376...   
1  [[-8.639847, 41.159826], [-8.640351, 41.159871...   
2  [[-8.612964, 41.140359], [-8.613378, 41.14035]...   
3  [[-8.574678, 41.151951], [-8.574705, 41.151942...   
4  [[-8.645994, 41.18049], [-8.645949, 41.180517]...   

                                        Tokenization  
0  [8a7b63adb347fff, 8a7b63adb35ffff, 8a7b63adbad...  
1  [8a7b63ad8047fff, 8a7b63ad8047fff, 8a7b63ad814...  
2  [8a7b63adb2cffff, 8a7b63adb2cffff, 8a7b63adb2e...  
3  [8a7b63370727fff, 8a7b63370727fff, 8a7b6337072...  
4  [8a7b63adea37fff, 8a7b63adea37fff, 8a7b63adea0...

__Conclusion :__

On a bien les données dans le format voulu.

# Séparation des trajectoires en début (input)/point(s) à prédire/destination

## Vérification de la tokenization

Le but est de vérifier la taille des listes de tokens avec la liste de points géographique correspondant.

In [13]:
def count_points(traj):
    return len(traj)

On vérifie le fonctionnement sur une ligne :

In [14]:
count_points(data_loaded['POLYLINE'][0])

23

In [15]:
data_loaded['POLYLINE'][0]

[[-8.618643, 41.141412],
 [-8.618499, 41.141376],
 [-8.620326, 41.14251],
 [-8.622153, 41.143815],
 [-8.623953, 41.144373],
 [-8.62668, 41.144778],
 [-8.627373, 41.144697],
 [-8.630226, 41.14521],
 [-8.632746, 41.14692],
 [-8.631738, 41.148225],
 [-8.629938, 41.150385],
 [-8.62911, 41.151213],
 [-8.629128, 41.15124],
 [-8.628786, 41.152203],
 [-8.628687, 41.152374],
 [-8.628759, 41.152518],
 [-8.630838, 41.15268],
 [-8.632323, 41.153022],
 [-8.631144, 41.154489],
 [-8.630829, 41.154507],
 [-8.630829, 41.154516],
 [-8.630829, 41.154498],
 [-8.630838, 41.154489]]

On définie une fonction ajoutant une colonne rescensant le nombre de points dans la trajectoire non tokenizée et une pour la trajectoire tokenizée.

In [16]:
def verif(data_load):
    data_csv=data_load
        
    data_csv['Nb_points'] = data_loaded['POLYLINE'].apply(lambda x: count_points(x))
    data_csv['Nb_points_token'] = data_loaded['Tokenization'].apply(lambda x: count_points(x))
        

    return data_csv     

Enfin, on définie une fonction qui compare les deux longueurs pour chaque trajectoire, et bon l'appelle.

In [17]:
def meme_longueur(data_load) : #retourne vrai si la liste de tokens et celle de points geographiques sont de même taille
    dataframe=verif(data_load)
    data_liste = (dataframe['Nb_points']==dataframe['Nb_points_token'])
    listebool=data_liste.to_numpy()
    return (not(False in listebool))

In [18]:
meme_longueur(data_loaded)

True

__Conclusion :__

On a bien les bonnes tailles de listes des points tokenisés.
On remarque aussi que les trajectoires ne sont pas toutes de même longueur.

On peut donc ne garder qu'une des deux colonnes et procéder à la suite.

In [20]:
data_loaded['Nb_points']=data_loaded['POLYLINE'].apply(lambda x: count_points(x))

In [21]:
data_loaded.head()

TRIP_ID CALL_TYPE  ORIGIN_CALL  ORIGIN_STAND   TAXI_ID  \
0  1372636858620000589         C          NaN           NaN  20000589   
1  1372637303620000596         B          NaN           7.0  20000596   
2  1372636951620000320         C          NaN           NaN  20000320   
3  1372636854620000520         C          NaN           NaN  20000520   
4  1372637091620000337         C          NaN           NaN  20000337   

    TIMESTAMP DAY_TYPE  MISSING_DATA  \
0  1372636858        A         False   
1  1372637303        A         False   
2  1372636951        A         False   
3  1372636854        A         False   
4  1372637091        A         False   

                                            POLYLINE  \
0  [[-8.618643, 41.141412], [-8.618499, 41.141376...   
1  [[-8.639847, 41.159826], [-8.640351, 41.159871...   
2  [[-8.612964, 41.140359], [-8.613378, 41.14035]...   
3  [[-8.574678, 41.151951], [-8.574705, 41.151942...   
4  [[-8.645994, 41.18049], [-8.645949, 41.180517]...   

                                        Tokenization  Nb_points  \
0  [8a7b63adb347fff, 8a7b63adb35ffff, 8a7b63adbad...         23   
1  [8a7b63ad8047fff, 8a7b63ad8047fff, 8a7b63ad814...         19   
2  [8a7b63adb2cffff, 8a7b63adb2cffff, 8a7b63adb2e...         65   
3  [8a7b63370727fff, 8a7b63370727fff, 8a7b6337072...         43   
4  [8a7b63adea37fff, 8a7b63adea37fff, 8a7b63adea0...         29   

   Nb_points_token  
0               23  
1               19  
2               65  
3               43  
4               29

In [22]:
#data_loaded.drop('Nb_points_token',axis=1)

TRIP_ID CALL_TYPE  ORIGIN_CALL  ORIGIN_STAND   TAXI_ID  \
0        1372636858620000589         C          NaN           NaN  20000589   
1        1372637303620000596         B          NaN           7.0  20000596   
2        1372636951620000320         C          NaN           NaN  20000320   
3        1372636854620000520         C          NaN           NaN  20000520   
4        1372637091620000337         C          NaN           NaN  20000337   
...                      ...       ...          ...           ...       ...   
1710665  1404171463620000698         C          NaN           NaN  20000698   
1710666  1404171367620000670         C          NaN           NaN  20000670   
1710667  1388745716620000264         C          NaN           NaN  20000264   
1710668  1404141826620000248         B          NaN          12.0  20000248   
1710669  1404157147620000079         B          NaN          34.0  20000079   

          TIMESTAMP DAY_TYPE  MISSING_DATA  \
0        1372636858        A         False   
1        1372637303        A         False   
2        1372636951        A         False   
3        1372636854        A         False   
4        1372637091        A         False   
...             ...      ...           ...   
1710665  1404171463        A         False   
1710666  1404171367        A         False   
1710667  1388745716        A         False   
1710668  1404141826        A         False   
1710669  1404157147        A         False   

                                                  POLYLINE  \
0        [[-8.618643, 41.141412], [-8.618499, 41.141376...   
1        [[-8.639847, 41.159826], [-8.640351, 41.159871...   
2        [[-8.612964, 41.140359], [-8.613378, 41.14035]...   
3        [[-8.574678, 41.151951], [-8.574705, 41.151942...   
4        [[-8.645994, 41.18049], [-8.645949, 41.180517]...   
...                                                    ...   
1710665  [[-8.612469, 41.14602], [-8.612487, 41.145993]...   
1710666  [[-8.610138, 41.140845], [-8.610174, 41.140935...   
1710667                                                 []   
1710668  [[-8.630712, 41.154885], [-8.63073, 41.154813]...   
1710669  [[-8.615538, 41.140629], [-8.615421, 41.140746...   

                                              Tokenization  Nb_points  
0        [8a7b63adb347fff, 8a7b63adb35ffff, 8a7b63adbad...         23  
1        [8a7b63ad8047fff, 8a7b63ad8047fff, 8a7b63ad814...         19  
2        [8a7b63adb2cffff, 8a7b63adb2cffff, 8a7b63adb2e...         65  
3        [8a7b63370727fff, 8a7b63370727fff, 8a7b6337072...         43  
4        [8a7b63adea37fff, 8a7b63adea37fff, 8a7b63adea0...         29  
...                                                    ...        ...  
1710665  [8a7b63adb297fff, 8a7b63adb297fff, 8a7b63adb29...         32  
1710666  [8a7b63375957fff, 8a7b63375957fff, 8a7b63adb2c...         30  
1710667                                                 []          0  
1710668  [8a7b63adb907fff, 8a7b63adb907fff, 8a7b63adb90...         62  
1710669  [8a7b63adb257fff, 8a7b63adb257fff, 8a7b63adb25...         27  

[1710670 rows x 11 columns]

In [23]:
data_loaded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1710670 entries, 0 to 1710669
Data columns (total 12 columns):
 #   Column           Dtype  
---  ------           -----  
 0   TRIP_ID          int64  
 1   CALL_TYPE        object 
 2   ORIGIN_CALL      float64
 3   ORIGIN_STAND     float64
 4   TAXI_ID          int64  
 5   TIMESTAMP        int64  
 6   DAY_TYPE         object 
 7   MISSING_DATA     bool   
 8   POLYLINE         object 
 9   Tokenization     object 
 10  Nb_points        int64  
 11  Nb_points_token  int64  
dtypes: bool(1), float64(2), int64(5), object(4)
memory usage: 158.2+ MB


## Troncature

Le problème que nous rencontrons est que BERT a pour entrée maximale 512 tokens. Or certaines trajectoires dépasse ce seuil. Afin de pouvoir utiliser BERT dans le cas de longues trajectoires, plusieurs s'offrent à nous. Une d'elle consiste à ne considérer que la fin de la trajectoire étudiée si elle est trop longue. C'est celle que nous mettons en oeuvre ici.

Nous avons besoin de connaitre la taille que prendra le contexte à concaténer pour rentrer dans la limite des 512 tokens après concaténation. Nous la fixons avec la variable "limite_entrée" à 400 pour le moment.

On commence par une étape d'étude de nos données : on regarde la proportion de trajectoire trop longues, les longueurs moyennes, ... 

In [24]:
## On commence par faire une copie de data_loaded au cas où on ferait des modifications non voulues de data_loaded et de perdre la bonne version
df=data_loaded.copy()

In [25]:
## On se donne une idée de la longueur max des trajectoires considérées

def recup_max(dataset, col):
    maximum=0
    for i in range(len(dataset[col])):
        if maximum<dataset[col][i]:
            maximum=dataset[col][i]
            position=i
    return [maximum,position]

#On peut rérer le max directement avec max(dataf['Nb_points']) mais cela donne pas la ligne.

In [26]:
maximum,position=recup_max(df, 'Nb_points')
maximum,position

(3881, 1093727)

In [29]:
"""
#Fonction pour avoir une idée du nombre/de la proportion de trajectoires trop longues

max_bert = 512

def compter_cb_trop_long(dataset,col):
    liste=[]
    for i in range(len(dataset[col])):
        if dataset[col][i]>max_bert:
            liste.append(i)
    return [len(liste),liste]

compteur,lis=compter_cb_trop_long(df,'Nb_points')
print(compteur,compteur/len(df.index))
    

"""


1947 0.001138150549200021


In [30]:
longueur_moyenne = df['Nb_points'].mean()
longueur_moyenne

48.758314578498485

In [31]:
stat.median(df['Nb_points'])

41.0

In [32]:
stat.mode(df['Nb_points'])

35

In [35]:
stat.pstdev(df['Nb_points'])

45.65371070883209

In [36]:
stat.pvariance(df['Nb_points'])

2084.2613014857293

In [37]:

limite_entree=400 #trajectoire maximale qu'on fixe (nb de jetons)


def troncature(datasetav,datasetap,limite_entree): # modifie le datasetap sans toucher au datasetav
#Retourne le nb de trajectoire modifiées

    nb=0
    for i in range(len(datasetav['Nb_points'])): # on parcourt les lignes de la colonne 'Nb_points'
        longueur=datasetav['Nb_points'][i]
        if longueur>limite_entree: #si trop long
            nb+=1 
            
            
            #Si on veut garder le début de la trajectoire :
            # datasetap['POLYLINE'][i]=datasetav['POLYLINE'][i][:limite_entree] 
            # datasetap['Tokenization'][i]=datasetav['Tokenization'][i][:limite_entree]    
            
            #Si on veut garder la fin de la trajectoire :
            indice_deb= longueur-limite_entree
            datasetap['POLYLINE'][i]=datasetav['POLYLINE'][i][indice_deb:] 
            datasetap['Tokenization'][i]=datasetav['Tokenization'][i][indice_deb:]    
            
            
    return nb

In [38]:
#On garde des copies
data_truncated=df.copy()

In [39]:
#On effectue la troncature
troncature(df,data_truncated,limite_entree)

C:\Users\fc891936\AppData\Local\Temp\ipykernel_27244\2526360032.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasetap['POLYLINE'][i]=datasetav['POLYLINE'][i][indice_deb:]
C:\Users\fc891936\AppData\Local\Temp\ipykernel_27244\2526360032.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasetap['Tokenization'][i]=datasetav['Tokenization'][i][indice_deb:]


3437

In [40]:
data_truncated.head()

TRIP_ID CALL_TYPE  ORIGIN_CALL  ORIGIN_STAND   TAXI_ID  \
0  1372636858620000589         C          NaN           NaN  20000589   
1  1372637303620000596         B          NaN           7.0  20000596   
2  1372636951620000320         C          NaN           NaN  20000320   
3  1372636854620000520         C          NaN           NaN  20000520   
4  1372637091620000337         C          NaN           NaN  20000337   

    TIMESTAMP DAY_TYPE  MISSING_DATA  \
0  1372636858        A         False   
1  1372637303        A         False   
2  1372636951        A         False   
3  1372636854        A         False   
4  1372637091        A         False   

                                            POLYLINE  \
0  [[-8.618643, 41.141412], [-8.618499, 41.141376...   
1  [[-8.639847, 41.159826], [-8.640351, 41.159871...   
2  [[-8.612964, 41.140359], [-8.613378, 41.14035]...   
3  [[-8.574678, 41.151951], [-8.574705, 41.151942...   
4  [[-8.645994, 41.18049], [-8.645949, 41.180517]...   

                                        Tokenization  Nb_points  \
0  [8a7b63adb347fff, 8a7b63adb35ffff, 8a7b63adbad...         23   
1  [8a7b63ad8047fff, 8a7b63ad8047fff, 8a7b63ad814...         19   
2  [8a7b63adb2cffff, 8a7b63adb2cffff, 8a7b63adb2e...         65   
3  [8a7b63370727fff, 8a7b63370727fff, 8a7b6337072...         43   
4  [8a7b63adea37fff, 8a7b63adea37fff, 8a7b63adea0...         29   

   Nb_points_token  
0               23  
1               19  
2               65  
3               43  
4               29

In [44]:
#on maj le nb de points dans la trajectoire

data_truncated['Nb_points'] = data_truncated['POLYLINE'].apply(lambda x: count_points(x))

In [42]:
max(data_truncated['Nb_points'])

400

On a obtenu une version avec les trajectoires tronquées pour ne pas avoir de trajectoires trop longues.

## Détection de trajectoires trop courtes
Le but est d'enlever des potentielles trajectoires de moins de 3 points.

In [46]:
data_truncated_copy=data_truncated.copy()

In [47]:
def liste_traj_trop_courtes(dataframe_a) :
    index_to_remove =[]
    for i in range(len(dataframe_a['POLYLINE'])):
        if dataframe_a['Nb_points'][i]<3:
            index_to_remove.append(i)
    return index_to_remove
    

In [48]:
liste_index_a_enlever=liste_traj_trop_courtes(data_loaded)

In [49]:
len(liste_index_a_enlever)

43904

In [50]:
liste_traj_trop_courtes(data_loaded) == liste_traj_trop_courtes(data_truncated)

True

In [51]:
len(liste_index_a_enlever)/len(data_truncated.index), len(liste_index_a_enlever)

(0.025664798003121585, 43904)

In [52]:
liste_index_a_enlever[0]

54

In [40]:
data_loaded.iloc[54].name

'54'

In [33]:
type(liste_index_a_enlever[0])

int

In [53]:

def to_string_with_cote (liste):
    for i in range(len(liste)):
        liste[i] = str(liste[i])

    return liste

In [60]:
to_string_with_cote ([1,2,3])

["'1'", "'2'", "'3'"]

In [108]:
def remove_traj_too_short(dataframe):
    list_traj_to_remove = liste_traj_trop_courtes(dataframe)
    list_traj_to_remove=to_string_with_cote(list_traj_to_remove)
    dataframe_new=dataframe.drop(index=list_traj_to_remove)
    
    return dataframe_new
    
        

In [109]:
data_clean=remove_traj_too_short(data_loaded)

In [110]:
data_clean_truncated=remove_traj_too_short(data_truncated)

In [57]:
len(data_clean.index)==len(data_loaded.index)-len(liste_index_a_enlever)

True

In [69]:
data_loaded.index

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '427657', '427658', '427659', '427660', '427661', '427662', '427663',
       '427664', '427665', '427666'],
      dtype='object', length=427667)

In [59]:
data_clean.iloc[54]

TRIP_ID                                          1372637230620000503
CALL_TYPE                                                          C
ORIGIN_CALL                                                      NaN
ORIGIN_STAND                                                     NaN
TAXI_ID                                                     20000503
TIMESTAMP                                                 1372637230
DAY_TYPE                                                           A
MISSING_DATA                                                   False
POLYLINE           [[-8.630568, 41.154795], [-8.63064, 41.154813]...
Tokenization       [8a7b63adb907fff, 8a7b63adb907fff, 8a7b63adb92...
Nb_points                                                         39
Nb_points_token                                                   39
Name: 55, dtype: object

In [60]:
# Serializing json
json_object = json.dumps(data_clean.to_dict())

# Writing to sample.json
with open("train_clean.json", "w") as outfile:
    outfile.write(json_object)

In [111]:
# Serializing json
json_object = json.dumps(data_clean_truncated.to_dict())

# Writing to sample.json
with open("train_clean_truncated.json", "w") as outfile:
    outfile.write(json_object)

## Définition des fonctions de séparation

Pour le moment, on ne veut prédire qu'un seul point.

In [33]:
def padding_et_sep(traj):
    destination = traj[-1]
    a_predire= traj[-2]
    deb=traj[:-2] #on enleve les deux derniers /!\ pas de = car pop modifie la liste et renvoie l'élément enlevé
    while len(deb)<limite_entree-2:
        deb.append('[PAD]')
    return (deb, a_predire, destination )
    


In [37]:
traj_input, traj_target, traj_dest = separation(dataf['POLYLINE'][0])

In [38]:
len(traj_input)

398

In [39]:
traj_input

[[-8.618643, 41.141412],
 [-8.618499, 41.141376],
 [-8.620326, 41.14251],
 [-8.622153, 41.143815],
 [-8.623953, 41.144373],
 [-8.62668, 41.144778],
 [-8.627373, 41.144697],
 [-8.630226, 41.14521],
 [-8.632746, 41.14692],
 [-8.631738, 41.148225],
 [-8.629938, 41.150385],
 [-8.62911, 41.151213],
 [-8.629128, 41.15124],
 [-8.628786, 41.152203],
 [-8.628687, 41.152374],
 [-8.628759, 41.152518],
 [-8.630838, 41.15268],
 [-8.632323, 41.153022],
 [-8.631144, 41.154489],
 [-8.630829, 41.154507],
 [-8.630829, 41.154516],
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '

In [40]:
traj_target , traj_dest

([-8.630829, 41.154498], [-8.630838, 41.154489])

In [204]:
df_to_load['traj_input'],df_to_load['traj_target'],df_to_load['traj_dest']=dataf['POLYLINE'].apply(lambda x: separation(x))

IndexError: list index out of range